In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=9329f61319190656638500632eb287528e32bb2b536522f2f874909405982335
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, sum, max, min, expr, month, date_format
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.appName("CovidDataAnalysis").getOrCreate()

In [4]:
df = spark.read.csv("/content/complete.csv", header=True, inferSchema=True)

In [5]:
df.show(5)

+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+
|      Date|Name of State / UT|Latitude|Longitude|Total Confirmed cases|Death|Cured/Discharged/Migrated|New cases|New deaths|New recovered|
+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+
|2020-01-30|            Kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|
|2020-01-31|            Kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|
|2020-02-01|            Kerala| 10.8505|  76.2711|                  2.0|    0|                      0.0|        1|         0|            0|
|2020-02-02|            Kerala| 10.8505|  76.2711|                  3.0|    0|                      0.0|        1|         0|            0|
|2020-02-03|        

  # 1.Convert all states names to lower case

In [6]:
df = df.withColumn("Name of State / UT", lower(col("Name of State / UT")))
df.show(5)

+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+
|      Date|Name of State / UT|Latitude|Longitude|Total Confirmed cases|Death|Cured/Discharged/Migrated|New cases|New deaths|New recovered|
+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+
|2020-01-30|            kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|
|2020-01-31|            kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|
|2020-02-01|            kerala| 10.8505|  76.2711|                  2.0|    0|                      0.0|        1|         0|            0|
|2020-02-02|            kerala| 10.8505|  76.2711|                  3.0|    0|                      0.0|        1|         0|            0|
|2020-02-03|        

# 2.The day with the greatest number of COVID cases

In [8]:
max_cases_day = df.groupBy("Date").agg(sum("New cases").alias("Total_New_Cases"))\
                  .orderBy(col("Total_New_Cases").desc()).first()
print(f"Date: {max_cases_day['Date']}, Total New Cases: {max_cases_day['Total_New_Cases']}")


Date: 2020-07-18, Total New Cases: 70962


# 3. Find the state with the second-largest number of COVID cases (Total Confirmed cases)

In [9]:
# Group by state and calculate the total confirmed cases for each state
state_cases = df.groupBy("Name of State / UT").agg(sum("Total Confirmed cases").alias("Total_Cases"))

# Order by total cases in descending order and collect the result
second_largest_state = state_cases.orderBy(col("Total_Cases").desc()).collect()[1]

# Print the result with the state name and total cases
print(f"State: {second_largest_state['Name of State / UT']}, Total Cases: {second_largest_state['Total_Cases']}")



State: tamil nadu, Total Cases: 7847083.0


# 4. Find the Union Territory with the least number of deaths (Death)


In [10]:
# Filter, group, and aggregate total deaths for specified Union Territories
union_territory_deaths = df.filter(df["Name of State / UT"].isin(["delhi", "puducherry", "lakshadweep",
                                                                  "ladakh", "daman and diu",
                                                                  "dadra and nagar haveli",
                                                                  "chandigarh", "andaman and nicobar islands"]))\
                           .groupBy("Name of State / UT").agg(sum("Death").alias("Total_Deaths"))

# Find the Union Territory with the least number of deaths
least_deaths_ut = union_territory_deaths.orderBy(col("Total_Deaths").asc()).first()

# Print the result
print(f"Union Territory: {least_deaths_ut['Name of State / UT']}, Total Deaths: {least_deaths_ut['Total_Deaths']}")


Union Territory: andaman and nicobar islands, Total Deaths: 64.0


# 5. Find the state with the lowest Death to Total Confirmed cases ratio

In [11]:
# Calculate the death-to-confirmed-cases ratio for each state/UT
state_death_ratio = df.groupBy("Name of State / UT").agg(
    (sum("Death") / sum("Total Confirmed cases")).alias("Death_Confirmed_Ratio")
)

# Find the state/UT with the lowest death-to-confirmed-cases ratio
lowest_death_ratio_state = state_death_ratio.orderBy(col("Death_Confirmed_Ratio").asc()).first()

# Print the result
print(f"State/UT: {lowest_death_ratio_state['Name of State / UT']}, Death-Confirmed Ratio: {lowest_death_ratio_state['Death_Confirmed_Ratio']}")


State/UT: union territory of ladakh, Death-Confirmed Ratio: 0.0


# 6. Find which month has the most newer recovered cases

In [12]:
# Extract month from date and calculate total recovered cases for each month
monthly_recovered_cases = df.withColumn("Month", date_format(col("Date"), "MMMM"))\
                            .groupBy("Month").agg(sum("New recovered").alias("Total_Recovered"))

# Find the month with the most recovered cases
most_recovered_month = monthly_recovered_cases.orderBy(col("Total_Recovered").desc()).first()

# Print the result
print(f"Month: {most_recovered_month['Month']}, Total Recovered Cases: {most_recovered_month['Total_Recovered']}")


Month: July, Total Recovered Cases: 722983
